# Lady Gaga Instagram Posts

## Libs

In [2]:
import pandas as pd
import matplotlib as plt
import seaborn as sns

from pathlib import Path


## DATA

In [11]:
current_path = Path.cwd()
data_dir = current_path.parent / 'data' / 'processed'
data = f'{data_dir}/instagram_ladygaga_2025-05-12_14-11-29_processed.csv'
df = pd.read_csv(data, sep=';')

In [12]:
df.head()

,post_id,caption,likes_count,media_url,date,time
0,3620880300369349579,And just like that we're back 🖤 VIVA LA MAYHEM...,2629615,NaN,28-04-2025,17:49
1,3625179909652482815,Nothing could prepare me for the feeling I had...,4345039,NaN,04-05-2025,16:12
